cgv.ipynb에서 리뷰만 제외한 것  
대신 감독/등장인물의 정보를 수집한다 

1. 관람포인트 수집  
#reviewSection > div.cnms01020_chartWrap__SMzm5 > p
<p class="cnms01020_chartSummary__4qMZm">배우연기가 메소드급인&nbsp;스트레스가 해소되는 영화</p>

2. 감독/등장인물  

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# 대상 URL
url = 'https://cgv.co.kr/cnm/cgvChart/movieChart/78000'

# Selenium WebDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

try:
    # 웹 페이지 접속
    driver.get(url)

    # '#detailSection' 요소가 나타날 때까지 최대 10초간 기다립니다.
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.ID, "detailSection")))

    # 페이지의 소스 코드를 가져와서 BeautifulSoup 객체 생성
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # 감독/등장인물 섹션 선택
    section_selector = '#detailSection > section:nth-child(1)'
    section_element = soup.select_one(section_selector)
    
    people_list = []
    if section_element:
        # 각 인물 정보를 담고 있는 div 태그들을 모두 찾습니다.
        people_elements = section_element.select(".characterScroll_listItem__AdQvD")
        
        for person in people_elements:
            # 역할(감독 등)과 이름 텍스트를 찾습니다.
            role = person.select_one(".characterScroll_chName__02_mk")
            name = person.select_one(".characterScroll_realName__ghkOZ")
            
            role_text = role.get_text(strip=True) if role else ""
            name_text = name.get_text(strip=True) if name else ""

            # 역할과 이름을 조합하여 리스트에 추가 (역할이 있는 경우에만)
            if role_text and name_text:
                people_list.append(f"{role_text} {name_text}")
            elif name_text:
                people_list.append(name_text)

        # 리스트의 요소들을 쉼표와 공백으로 연결하여 출력
        result_string = ", ".join(people_list)
        print(result_string)
        
    else:
        print(f"'{section_selector}'에 해당하는 요소를 찾을 수 없습니다.")

finally:
    # 작업이 끝나면 브라우저를 닫습니다.
    driver.quit()

/Users/swoo64/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


감독 채드 스타헬스키, 감독 데이빗 레이치, 키아누 리브스, 알피 알렌, 윌렘 데포, 아드리안 팔리키, 브리짓 모이나한


# 웹크롤링

In [ ]:
# 필요한 라이브러리들을 가져옵니다.
import pandas as pd
import os
import re
import time
from tqdm import tqdm # 진행 상황을 시각적으로 보여주기 위한 라이브러리

# 셀레니움 관련 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# 웹드라이버 자동 설치를 위한 라이브러리
from webdriver_manager.chrome import ChromeDriverManager

# --- 설정 ---
# 몇 번째 영화부터 크롤링을 시작할지 CSV의 '인덱스' 기준으로 지정합니다. (기본값: 1)
# 예를 들어 11로 설정하면, 인덱스 1~10번 영화는 건너뛰고 11번 영화부터 시작합니다.
START_FROM_INDEX = 1

# --- 0. 크롬 드라이버 설정 ---
# GoogleBot으로 User-Agent를 설정합니다.
chrome_options = Options()
chrome_options.add_argument("user-agent=GoogleBot")
# 백그라운드에서 실행하려면 아래 주석을 해제하세요.
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--log-level=3") # 콘솔 로그를 최소화합니다.
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

# 크롬 드라이버를 최신 버전으로 자동 설치하고 서비스를 시작합니다.
try:
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    # 웹 페이지의 요소가 나타날 때까지 최대 10초간 기다리는 WebDriverWait 객체를 생성합니다.
    wait = WebDriverWait(driver, 10)
    print("✅ 웹 드라이버 설정이 완료되었습니다.")
except Exception as e:
    print(f"❌ 웹 드라이버 설정 중 오류가 발생했습니다: {e}")
    print("인터넷 연결을 확인하거나, Chrome 브라우저가 설치되어 있는지 확인해주세요.")
    driver = None # 드라이버 설정 실패 시 이후 코드 실행 방지

# --- 1. 크롤링할 영화 목록 불러오기 ---
if driver:
    try:
        movie_list_df = pd.read_csv("../../영화관람객크롤링/영화목록.csv", encoding="utf-8-sig")
        # '인덱스'와 '영화명' 컬럼이 있는지 확인
        if '인덱스' not in movie_list_df.columns or '영화명' not in movie_list_df.columns:
            raise ValueError("'인덱스' 또는 '영화명' 컬럼을 찾을 수 없습니다. CSV 파일을 확인해주세요.")
        print(f"✅ 총 {len(movie_list_df)}개의 영화 목록을 불러왔습니다.")
    except FileNotFoundError:
        print("❌ '영화목록.csv' 파일을 찾을 수 없습니다. 파일 경로(../../영화관람객크롤링/영화목록.csv)를 확인해주세요.")
        movie_list_df = None # 파일이 없으면 None으로 초기화하여 에러 방지
    except ValueError as e:
        print(f"❌ CSV 파일 컬럼 오류: {e}")
        movie_list_df = None

# --- 13. 결과 저장 폴더 생성 ---
if driver and movie_list_df is not None:
    output_dir = "../영화리뷰데이터_저장/cgv데이터"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"✅ '{output_dir}' 폴더를 생성했습니다.")

# --- 메인 크롤링 로직 ---
if driver and movie_list_df is not None:
    # iterrows()를 사용하여 데이터프레임을 한 줄씩 순회합니다.
    for _, row in tqdm(movie_list_df.iterrows(), total=movie_list_df.shape[0], desc="전체 영화 크롤링 진행률"):
        movie_index = row['인덱스']
        movie_title = row['영화명']

        # 시작 인덱스 체크
        if movie_index < START_FROM_INDEX:
            continue # 시작 인덱스보다 작으면 현재 영화는 건너뜁니다.

        print(f"\n[{movie_index}/{len(movie_list_df)}] '{movie_title}' 크롤링 시작...")

        # 크롤링한 데이터를 저장할 변수 초기화
        movie_data = {}
        egg_index = pd.NA
        gender_dist = pd.NA
        age_dist = pd.NA
        director_cast = pd.NA # [추가] 감독/등장인물 변수
        viewing_point = pd.NA
        review_count = pd.NA

        try:
            # 2. CGV 영화 검색 페이지로 이동
            driver.get("https://cgv.co.kr/tme/itgrSrch")

            # 3. 검색어 입력
            search_box = wait.until(EC.presence_of_element_located((By.ID, "swrd")))
            search_box.clear()
            search_box.send_keys(movie_title)

            # [수정] 로딩 오버레이가 사라질 때까지 기다립니다.
            # 이 코드는 검색 버튼을 클릭하기 전에 나타나는 로딩 화면을 기다려 'ElementClickInterceptedException' 오류를 방지합니다.
            try:
                loading_overlay_selector = "div.loading_pageContainer__fvLY_"
                wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, loading_overlay_selector)))
            except TimeoutException:
                # 로딩 오버레이가 나타나지 않거나 이미 사라진 경우, 정상적으로 진행합니다.
                pass


            # 4. 검색 버튼 클릭
            search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-sch")))
            search_button.click()

            # 5. '영화' 탭 클릭
            try:
                short_wait = WebDriverWait(driver, 2)
                movie_tab = short_wait.until(EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='영화']]")))
                movie_tab.click()
            except TimeoutException:
                print(f"-> '{movie_title}' 검색 결과에서 '영화' 탭을 찾을 수 없습니다. 다음 영화로 넘어갑니다.")
                time.sleep(0.5)
                continue

            # 7. 첫 번째 검색 결과 클릭
            try:
                first_movie_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.tmes01040_section__D67K1 ul > li:first-child > button")))
                first_movie_link.click()
            except TimeoutException:
                print(f"-> '{movie_title}' 영화 목록을 찾을 수 없습니다. 다음 영화로 넘어갑니다.")
                continue

            # --- 영화 상세 페이지 데이터 추출 ---
            # 8. 에그 지수 추출
            try:
                egg_element = wait.until(EC.presence_of_element_located((By.XPATH, "//dt[contains(.,'에그지수')]/following-sibling::dd")))
                egg_text = egg_element.text
                egg_index = re.search(r'(\d+)', egg_text).group(1) if '%' in egg_text else pd.NA
                print(f"-> 에그 지수: {egg_index}%")
            except (TimeoutException, AttributeError):
                print("-> 에그 지수를 찾을 수 없습니다.")

            # 성별 예매 분포 추출
            try:
                short_wait = WebDriverWait(driver, 0.3)
                gender_element = short_wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.cnms01020_chartSection__00pmr > div:nth-child(2) > div:nth-child(1) > div > span")))
                gender_dist = gender_element.get_attribute("textContent").strip()
                print(f"-> {gender_dist}")
            except TimeoutException:
                print("-> 성별 예매 분포 정보를 찾을 수 없습니다.")

            # 연령별 예매 분포 추출
            try:
                short_wait = WebDriverWait(driver, 0.3)
                age_element = short_wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.cnms01020_chartSection__00pmr > div:nth-child(2) > div:nth-child(2) > div > span")))
                age_dist = age_element.get_attribute("textContent").strip()
                print(f"-> {age_dist}")
            except TimeoutException:
                print("-> 연령별 예매 분포 정보를 찾을 수 없습니다.")

            # [추가] 감독/등장인물 수집
            try:
                people_list = []
                # 인물 정보가 로드될 때까지 잠시 기다립니다.
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".characterScroll_listItem__AdQvD")))
                people_elements = driver.find_elements(By.CSS_SELECTOR, ".characterScroll_listItem__AdQvD")
                
                for person in people_elements:
                    # find_elements를 사용하여 특정 요소가 없는 경우에도 오류가 발생하지 않도록 합니다.
                    role_elements = person.find_elements(By.CSS_SELECTOR, ".characterScroll_chName__02_mk")
                    name_elements = person.find_elements(By.CSS_SELECTOR, ".characterScroll_realName__ghkOZ")
                    
                    role_text = role_elements[0].text.strip() if role_elements else ""
                    name_text = name_elements[0].text.strip() if name_elements else ""

                    if role_text and name_text:
                        people_list.append(f"{role_text} {name_text}")
                    elif name_text:
                        people_list.append(name_text)

                if people_list:
                    director_cast = ", ".join(people_list)
                    print(f"-> 감독/등장인물: {director_cast}")
                else:
                    # people_elements는 찾았지만 리스트가 비어있는 경우
                    print("-> 감독/등장인물 정보를 찾을 수 없습니다.")
            except TimeoutException:
                # 인물 정보 섹션 자체가 없는 경우
                print("-> 감독/등장인물 정보를 찾을 수 없습니다.")

            # 스크롤 로직을 먼저 실행하여 'lazy loading' 콘텐츠를 로드
            try:
                review_section_selector = "#reviewSection"
                review_section = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, review_section_selector)))
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", review_section)
                time.sleep(0.5) # 스크롤 후 데이터가 로드될 시간을 줌
            except TimeoutException:
                print("-> 페이지 하단 정보 영역(reviewSection)을 찾을 수 없어 스크롤하지 못했습니다.")

            # 관람포인트 수집
            try:
                viewing_point_selector = "p.cnms01020_chartSummary__4qMZm"
                short_wait = WebDriverWait(driver, 0.1)
                viewing_point_element = short_wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, viewing_point_selector)))
                viewing_point = viewing_point_element.get_attribute("textContent").strip().replace('\xa0', ' ')
                print(f"-> 관람포인트: {viewing_point}")
            except TimeoutException:
                print("-> 관람포인트 정보를 찾을 수 없습니다.")
            
            # 실관람평 개수 수집
            review_count = '0' # 리뷰가 없는 경우를 대비해 기본값을 '0'으로 설정
            try:
                # '등록된 실관람평이 없습니다.' 메시지가 있는지 먼저 확인
                try:
                    short_wait = WebDriverWait(driver, 0.1) # 짧게 기다림
                    short_wait.until(EC.presence_of_element_located((By.XPATH, "//p[contains(text(), '등록된 실관람평이 없습니다.')]")))
                    print("-> 등록된 실관람평이 없습니다.")
                    review_count = '0'
                except (NoSuchElementException, TimeoutException):
                    # 메시지가 없으면 h2 태그에서 개수 추출
                    review_count_selector = "div.cnms01020_reviewArea__52mah div.cnms01020_titleWrap__6t8uh > h2"
                    wait.until(
                        lambda d: re.search(r'\d', d.find_element(By.CSS_SELECTOR, review_count_selector).get_attribute("textContent")),
                        "실관람평 개수(숫자)가 표시될 때까지 대기"
                    )
                    review_count_element = driver.find_element(By.CSS_SELECTOR, review_count_selector)
                    review_count_text = review_count_element.get_attribute("textContent")
                    review_count = re.sub(r'\D', '', review_count_text)
                    print(f"-> 실관람평 개수: {review_count}")

            except TimeoutException:
                 print("-> 실관람평 개수 정보를 찾을 수 없습니다.")
                 review_count = pd.NA # 스크롤이나 영역 자체를 못찾는 예외 케이스
            
            # 수집한 데이터를 딕셔너리에 저장
            movie_data = {
                '영화제목': movie_title,
                '에그지수': egg_index,
                '성별': gender_dist,
                '연령별': age_dist,
                '감독/등장인물': director_cast, # [추가]
                '관람포인트': viewing_point,
                '실관람평개수': review_count
            }

        except Exception as e:
            print(f"❌ '{movie_title}' 크롤링 중 예상치 못한 오류 발생: {e}")
            continue

        # --- 추출한 데이터를 CSV 파일로 저장 ---
        if movie_data:
            df = pd.DataFrame([movie_data])
            file_path = os.path.join(output_dir, f"{movie_index}.csv")
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"✅ '{movie_title}' 데이터 저장 완료: {file_path}")
        else:
            print(f"-> '{movie_title}'에 대한 데이터를 저장하지 못했습니다.")

    # 모든 작업이 끝나면 드라이버 종료
    if driver:
        driver.quit()
    print("\n🎉 모든 영화에 대한 크롤링 작업이 완료되었습니다.")



✅ 웹 드라이버 설정이 완료되었습니다.
✅ 총 42700개의 영화 목록을 불러왔습니다.


전체 영화 크롤링 진행률:  77%|███████▋  | 32860/42700 [00:00<00:00, 66495.78it/s]


[38727/42700] '언더 유어 베드' 크롤링 시작...
-> 에그 지수: 65%
-> 성별예매 분포: 남 56%, 여 44%
-> 연령별예매 분포: 10대 0%, 20대 10%, 30대 26%, 40대 32%, 50대 33%
-> 감독/등장인물: 감독 사부, 이윤우, 이지훈, 신수항
-> 관람포인트: 배우연기가 메소드급인 몰입하여 보게 되는 영화
-> 실관람평 개수: 1267
✅ '언더 유어 베드' 데이터 저장 완료: ../영화리뷰데이터_저장/cgv데이터/38727.csv

[38728/42700] '제75회 작은영화영화제: 그 곳에는 잊을 수 없는 추억이 있다!' 크롤링 시작...
-> '제75회 작은영화영화제: 그 곳에는 잊을 수 없는 추억이 있다!' 검색 결과에서 '영화' 탭을 찾을 수 없습니다. 다음 영화로 넘어갑니다.

[38729/42700] '깨끗한 물속의 칼' 크롤링 시작...


전체 영화 크롤링 진행률:  77%|███████▋  | 32860/42700 [00:15<00:00, 66495.78it/s]

-> '깨끗한 물속의 칼' 검색 결과에서 '영화' 탭을 찾을 수 없습니다. 다음 영화로 넘어갑니다.


...

-> 실관람평 개수: 3
✅ '내가 가진 것은 아무것도 없었다' 데이터 저장 완료: ../영화리뷰데이터_저장/cgv데이터/42698.csv

[42699/42700] 'BIAF2025 기획상영 오늘봄' 크롤링 시작...
-> 'BIAF2025 기획상영 오늘봄' 검색 결과에서 '영화' 탭을 찾을 수 없습니다. 다음 영화로 넘어갑니다.


전체 영화 크롤링 진행률: 100%|█████████▉| 42699/42700 [5:32:25<00:07,  7.38s/it]


[42700/42700] '새엄마 물 맛-너무 맛있어요 새엄마 무삭제판' 크롤링 시작...
-> '새엄마 물 맛-너무 맛있어요 새엄마 무삭제판' 검색 결과에서 '영화' 탭을 찾을 수 없습니다. 다음 영화로 넘어갑니다.


전체 영화 크롤링 진행률: 100%|██████████| 42700/42700 [5:32:29<00:00,  2.14it/s]



🎉 모든 영화에 대한 크롤링 작업이 완료되었습니다.
